In [1]:
import os
import pandas as pd
import numpy as np
import fitsio
import cudf
from dask.distributed import Client
from dask import delayed

In [157]:
data_dir = os.environ['SPIKESDATA']
df = pd.read_parquet(os.path.join(data_dir, 'spikes_df_2010.parquet'), engine='pyarrow')
# spikes_db2 = spikes_db.set_index(['GroupNumber', 'Time'])
len(df)


11629115

In [203]:
gdf = cudf.DataFrame.from_pandas(df)

In [149]:
df_1d = df[(df['Time'] > '2010-05-13 00:00:00') & (df['Time'] < '2010-05-14 00:00:00')]
df_30d = df[(df['Time'] > '2010-05-13 00:00:00') & (df['Time'] < '2010-06-13 00:00:00')]
len(df_1d),  len(df_30d)

(49804, 1539077)

In [201]:
df.memory_usage(deep=True).sum() / (1024**2)

1552.6544380187988

In [144]:
df_1d.memory_usage(deep=True).sum()/(1024**2)

7.028678894042969

In [96]:
df_30d.memory_usage(deep=True).sum()/(1024**2)

215.60576248168945

In [206]:
%timeit df2 = df.drop_duplicates(['GroupNumber', 'Wavelength'])

1.71 s ± 38.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [209]:
%timeit gdf2 = gdf.drop_duplicates(['GroupNumber', 'Wavelength'])

400 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [156]:
dropped = df[~df.index.isin(df2.index)]

15455

In [158]:
len(dropped), len(dropped)/len(df) * 100

(15455, 0.13289919310282855)

In [154]:
dropped.head()

,Path,Size,Time,Wavelength,GroupNumber
292700,2010/05/18/2010-05-18T22:35:35.50Z_0171.spikes...,48960,2010-05-18 22:35:35.500000+00:00,171,42777
292705,2010/05/18/2010-05-18T22:35:46.21Z_0211.spikes...,100800,2010-05-18 22:35:46.210000+00:00,211,42778
292710,2010/05/18/2010-05-18T22:35:58.71Z_0304.spikes...,187200,2010-05-18 22:35:58.710000+00:00,304,42779
292711,2010/05/18/2010-05-18T22:35:59.57Z_0335.spikes...,334080,2010-05-18 22:35:59.570000+00:00,335,42779
292724,2010/05/18/2010-05-18T22:36:35.49Z_0171.spikes...,48960,2010-05-18 22:36:35.490000+00:00,171,42782


In [155]:
df.loc[df['GroupNumber']==42777]

,Path,Size,Time,Wavelength,GroupNumber
292696,2010/05/18/2010-05-18T22:35:25.47Z_0171.spikes...,51840,2010-05-18 22:35:25.470000+00:00,171,42777
292697,2010/05/18/2010-05-18T22:35:26.21Z_0211.spikes...,100800,2010-05-18 22:35:26.210000+00:00,211,42777
292698,2010/05/18/2010-05-18T22:35:28.71Z_0304.spikes...,184320,2010-05-18 22:35:28.710000+00:00,304,42777
292699,2010/05/18/2010-05-18T22:35:29.58Z_0335.spikes...,331200,2010-05-18 22:35:29.580000+00:00,335,42777
292700,2010/05/18/2010-05-18T22:35:35.50Z_0171.spikes...,48960,2010-05-18 22:35:35.500000+00:00,171,42777


In [198]:
%timeit mask_incomplete = df2.groupby('GroupNumber')['Wavelength'].count() != 7

323 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [212]:
incomplete_groups = mask_incomplete[mask_incomplete].index.values

In [213]:
incomplete_groups

array([   6698,    6783,   20125, ..., 1669502, 1676697, 1676702])

In [214]:
incomplete_idx = df2.loc[df2['GroupNumber'].isin(incomplete_groups)].index

In [215]:
df3 = df2.drop(incomplete_idx)

In [218]:
df3.head()

,Path,Size,Time,Wavelength,GroupNumber
0,2010/05/13/2010-05-13T00:00:02.09Z_0193.spikes...,106560,2010-05-13 00:00:02.090000+00:00,193,0
1,2010/05/13/2010-05-13T00:00:03.57Z_0094.spikes...,103680,2010-05-13 00:00:03.570000+00:00,94,0
2,2010/05/13/2010-05-13T00:00:05.07Z_0335.spikes...,126720,2010-05-13 00:00:05.070000+00:00,335,0
3,2010/05/13/2010-05-13T00:00:06.58Z_0171.spikes...,40320,2010-05-13 00:00:06.580000+00:00,171,0
4,2010/05/13/2010-05-13T00:00:08.08Z_0211.spikes...,60480,2010-05-13 00:00:08.080000+00:00,211,0


In [217]:
len(df3)

11442690

In [216]:
(len(df) - len(df3))/len(df) * 100

1.6030884551403957

In [140]:
%timeit dup30d = df_30d.groupby('GroupNumber')['Wavelength'].nunique() != 7

101 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [141]:
%timeit dup30d = df_30d.drop_duplicates(['GroupNumber', 'Wavelength']).groupby('GroupNumber')['Wavelength'].count() !=7

141 ms ± 614 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [105]:
gdf_1d = cudf.DataFrame.from_pandas(df_1d)

In [106]:
gdf_30d = cudf.DataFrame.from_pandas(df_30d)

In [136]:
dup1d = gdf_1d.drop_duplicates(['GroupNumber', 'Wavelength']).groupby('GroupNumber')['Wavelength'].count() !=7

In [139]:
%timeit dup30d = gdf_30d.drop_duplicates(['GroupNumber', 'Wavelength']).groupby('GroupNumber')['Wavelength'].count() !=7

67.1 ms ± 181 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [145]:
u_gdf30d = gdf_30d.drop_duplicates(['GroupNumber', 'Wavelength'])

NameError: name 'gdf' is not defined

### Get the filepaths (typically 7) for a given group

In [4]:
################################################################################################
# Pre-compute the 8-connectivity lookup table. This will be shared across parallel workers.
################################################################################################
# List of relative 2D coordinates for 8-neighbour connectiviy (9-element list). 1st one is the origin pixel.
coords_8nb = np.array([[0, 0], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1]])
# Array of 2D coordinates for a 4096 x 4096 array. Matrix convention is kept. [rows, cols] = [y-axis, x-axis]
ny, nx = [4096, 4096]
coords_1d = np.arange(nx * ny)
coordy, coordx = np.unravel_index(coords_1d, [ny, nx]) # also possible by raveling a meshgrid() output
coords2d = np.array([coordy, coordx])
# Create the array of 2D coordinates of 8-neighbours associated with each pixel.
# pixel 0 has 8 neighbour + itself, pixel 1 has 8 neighbour + itself, etc...
coords2d_8nb = coords2d[np.newaxis, ...] + coords_8nb[..., np.newaxis]
# Handle off-edges coordinates by clipping to the edges, operation done in-place. Here, square detector assumed. Update
# to per-axis clipping if that ever changes for another instrument.
np.clip(coords2d_8nb, 0, nx-1, out=coords2d_8nb)
# Convert to 1D coordinates.
index_8nb = np.array([coords2d_8nb[i, 0, :] * nx + coords2d_8nb[i, 1, :] for i in range(len(coords_8nb))],
                     dtype='int32', order='C').T
index_8nb.shape

(16777216, 9)

In [208]:
group_n = 0
fpaths = spikes_db2.loc[group_n]['Path'].values
spikes_list = [fitsio.read(os.path.join(data_dir, f)) for f in fpaths]

# nspikes = 0
# for spikes in spikes_list:
#     nspikes += spikes.shape[1]
# print('\ntotal spikes = ', nspikes)

4.65 ms ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [218]:
start_date = pd.Timestamp('2010-05-13 00:00:00')
end_date = pd.Timestamp('2010-05-13 23:59:59')

In [206]:
%timeit df0 = extract_all_coincidentals(spikes_list)

878 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
print(df0.head())
print(len(df0))

   coords  int1  int2  wref  w0  w1  w2  w3  w4  w5  w6
0   18917   122    11     0   1   1   0   0   0   0   0
1   19192   124     7     0   1   0   0   1   0   0   0
2   23013    75    10     0   1   1   0   0   0   0   0
3   23287   157     9     0   1   0   0   1   0   0   0
4   27109    38     9     0   1   1   0   0   0   0   0
16339
